In [ ]:
import pandas as pd
import random
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)

## url películas

In [ ]:
with open ('AccionDrama.json', 'r') as fichero:
    busqueda = json.load(fichero)

In [ ]:
lista_url = []

In [ ]:
for pelicula in busqueda:
    lista_url.append(f'https://www.imdb.com/title/{pelicula[0]}/fullcredits/?ref_=tt_ql_1')

## id actores en imdb

In [ ]:
lista_peliculas_y_actores = []
lista_todos_id = []

In [ ]:
for url in lista_url:
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()

    # cookies
    try:
        driver.find_element('css selector', '#imdb-bmo-footer > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
    except:
        pass

    sleep(1)

    id_actores_por_peli = [url]
    lista_href = []

    for actor in range (2, 12):
        xpath_actor = driver.find_element('xpath', f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{actor}]/td[2]/a')
        href_actor = xpath_actor.get_attribute('href')
        print(f'Voy por href: {href_actor}')
        lista_href.append(href_actor)

    for href in lista_href:
        patron_id = r'/name/(\w+)/'
        if 'https' in href:
            resultado = re.search(patron_id, href)
            id = resultado.group(1)
            print(f'Voy por id: {id}')
            id_actores_por_peli.append(id)
            lista_todos_id.append(id)
    
    driver.close()        
    
    lista_peliculas_y_actores.append(id_actores_por_peli)

## llamada a la api de actores

In [ ]:
json_info_actores = []

In [ ]:
contador = 0
for id in lista_todos_id:
    contador += 1

    url = f"https://moviesdatabase.p.rapidapi.com/actors/{id}"
    headers = {
        "X-RapidAPI-Key": "{vuestra API-Key}",
        "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers)
    print(f'({contador} - Actor {id}: {response.status_code}')

    json_info_actores.append(response.json())

### actores awards


In [ ]:
df_actores = pd.read_csv("../Fase3_datos/Archivos_definitivos/Actores.csv")
v_actores = df_actores.to_numpy().transpose().tolist()
actores = v_actores[1]
actores

In [ ]:
driver = webdriver.Chrome()

contador = 0
todos_actores_awards = []
for id in actores:
    contador +=1
    print(f'Actor {contador}/{(len(actores))}')
    actor_awards = [id,]
    try:
        
        driver.get(f'https://m.imdb.com/name/{id}/awards/')
        sleep(1)
        try:
            driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
        except:
            pass                                    
        
        list_raw_awards = []
        for child in range(1, 100):
            try:
                premio = driver.find_element('css selector', f'#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child({child})').text.split('\n')
                                                                
                list_raw_awards.append(premio)
            except:
                pass
        
        actor_awards.append(list_raw_awards)
    
    except:
        pass

    todos_actores_awards.append(actor_awards)

In [ ]:
patron = r'Ganador\s(.*)'

todos_actores_limpio = [] 

for lista in todos_actores_awards:

    actor_premios = [lista[0]]

    for elemento in lista[1]:
        
        for indice, linea in enumerate(elemento):

            if 'Ganador' in linea:
                resultado = re.search(patron, linea)
                titulo = resultado.group(1)
                premio_completo = f'{elemento[0]}, {titulo}'
                actor_premios.append(premio_completo)
    
    todos_actores_limpio.append(actor_premios)

In [ ]:
lista_final = []

In [ ]:
for lista in todos_actores_limpio:
    if len(lista) == 1:
        lista.append(np.nan)

# lista_final = []

for lista in todos_actores_limpio:
    
    for elemento in lista[1:]:
        lista_intermedia = [lista[0],]
        lista_intermedia.append(elemento)
        lista_final.append(lista_intermedia)